In [1]:
import torch
from llm.create import get_send_fn, get_text_emb_send_fn
from search.searcher import Searcher
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from lc_helper.wrapper import LcChatWrapper
from index.vectorstore import VectorStoreIndexer

print(torch.cuda.is_available())

True

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
### Prameters to play with ###
llm_source = "huggingface"
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
vectorstore_dir= "./index_results/atelectasis/vectorstore"
##############################

In [ ]:
vectorstore = VectorStoreIndexer.load_db(vectorstore_dir)
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = LcChatWrapper(send_fn=get_send_fn(soure=llm_source, model_name=model_name))
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
res = rag_chain.invoke("What are symptoms of atelectasis?")

In [ ]:
print(res)